<img src="logo.png" width="420px">

# `TODOIST AUTOMATION`

### Settings

#### Libraries

In [88]:
from todoist_api_python.api import TodoistAPI
from datetime import datetime, timedelta, timezone
import functions as fun
import random
import json

#### Dependences

In [89]:
import credentials

#### API

In [90]:
api_token = credentials.api_token
api = TodoistAPI(api_token)
headers = {
    "Content-Type": "application/json",
    "X-Request-Id": "un_valor_único_por_cada_solicitud",
    "Authorization": f"Bearer {api_token}"
}

### `Data`

#### Projects

In [91]:
all_projects = fun.getProjects()
projects_dict_id, projects_dict_name = fun.getProjectsDicts(all_projects)

#### Tasks

In [92]:
def refreshTasks():
    all_tasks = fun.getTasks()
    task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
    return all_tasks, task_dict_id, task_dict_name

In [93]:
all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### Sections

In [94]:
all_sections = fun.getSections()
sections_dict_id, sections_dict_name = fun.getSectionsDicts(all_sections)

#### Test

In [95]:
index = random.randint(0,len(all_tasks))
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']], '-')
except TypeError:
    print(all_tasks[index]['content'], "-", "No date" , "-",projects_dict_id[all_tasks[index]['project_id']], '-')

Summaries - No date - Cursos -


#### Labels

In [96]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

['Med',
 'PC',
 'Phone',
 'Short',
 'Home',
 'Long',
 'Outside',
 'Recurring',
 '❌_streak',
 '✅_streak',
 'Vacations']

#### F1 Calendar

In [97]:
def getF1Calendar():
    return True

### `Orders`

#### Creating tasks

In [98]:
def createTask(
    assignee_id = None,
    comment_count = 0,
    is_completed = False,
    content = "No se añadio título",
    description = None,
    due = None,
    duration = None,
    labels = [],
    order = 1,
    priority = 4,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            content = content,
            description = description,
            due = due,
            duration = duration,
            labels = labels,
            order = order,
            priority = fun.priorityInversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
        )
        print(f'Task "{task.content}" ({task.id}) was created correctly')
    except Exception as error:
        print(error)

In [99]:
# createTask(content='Prueba',
#             labels=['Short'],
#             priority=1,
#             due={"string":"today"}
# )
# all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### Editing tasks

In [100]:
def editTask(
    task_id,
    _content = None,
    _assignee_id = None,
    _description = None,
    _due_string = None,
    _due_date = None,
    _due_datetime = None,
    _labels = None,
    _priority = None,
):
    task = fun.getTask(task_id)
    content = task.content if _content==None else _content
    assignee_id = task.assignee_id if _assignee_id==None else _assignee_id
    description = task.description if _description==None else _description
    due_string = task.due.string if _due_string==None else _due_string
    due_date = task.due.date if _due_date==None else _due_date
    due_datetime = task.due.datetime if _due_datetime==None else _due_datetime
    labels = _labels if _labels!=None else task.labels
    priority = task.priority if _priority==None else _priority
    
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        api.update_task(
            task_id = task_id,
            assignee_id = assignee_id,
            content = content,
            description = description,
            labels = labels,
            priority = priority,
            _due_string = due_string,
            _due_date = due_date,
            _due_datetime = due_datetime
        )
        print(f'Task {content} updated correctly')
    except Exception as error:
        print(error)

#### Add duration label

In [101]:
def updateTaskDurationLabel(task_id, task_duration):
    duration_label = fun.getDurationLabel(task_duration)
    task_id = task_id[0]
    task_labels_without_duration = fun.getLabelsWithoutDuration(task_id)
    task_labels_without_duration.append(duration_label)
    editTask(task_id=task_id,
             _labels=task_labels_without_duration)

In [102]:
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
to_update = False

for task in all_tasks:
    if task['duration'] != None and task['due'] != None:
        if fun.getDurationLabel(task['duration']['amount']) not in task['labels']:
            to_update = True
            updateTaskDurationLabel([task['id']], task['duration']['amount'])

if not to_update:
    print('No task to be updated')
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No task to be updated


#### Capitalize titles

In [103]:
to_update = False
for task in all_tasks:
    if task["content"][0].upper() != task["content"][0]:
        to_update = True
        task_id = task["id"]
        editTask(task_id = task_id,
            _content = task["content"][0].upper() + task["content"][1:])

if not to_update:
    print("No tasks to be updated")
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to be updated


#### Uncomplete recurring tasks

In [104]:
with open('recurringtasksdiego', 'r') as archivo_json:
    recurringtasks = json.load(archivo_json)

In [105]:
to_update = False
for task_id in recurringtasks.keys():
    task = fun.getTask(task_id)
    if task.is_completed == True:
        to_update = True
        fun.uncompleteTask(task_id)
        if 'Vacations' in task.labels:
            editTask(task_id=task_id,
                     _due_date="", _due_string="Today", _due_datetime="") #no pilla bien las fechas
if not to_update:
    print("No tasks to be updated")
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to be updated


#### Export all tasks and it's status

In [106]:
tasks = api.get_tasks()
recurringtasks={}
for task in tasks:
    if 'Recurring' in task.labels:
        recurringtasks[task.id]=task.is_completed
with open('recurringtasksdiego', 'w') as file:
    json.dump(recurringtasks, file)